Setup environment

In [1]:
# Ensure src folder is importable
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Auto-reload changes in .py files
%load_ext autoreload
%autoreload 2

Imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import matplotlib.pyplot as plt

from src.data.data_loader import CryptoDataLoader
from src.data.feature_engineering import FeatureEngineer
from src.models.lstm import LSTMModel

Load & prepare data

In [3]:
# Load BTC data
loader = CryptoDataLoader(data_dir="data/raw")
df = loader.load_saved_data("BTC-USD", "2024-01-01", "2025-01-01")

In [4]:
# Feature engineering
fe = FeatureEngineer()
df_features = fe.add_technical_indicators(df)

j:\Data Science\crypto_forecasting\venv\Lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
j:\Data Science\crypto_forecasting\venv\Lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


In [5]:
# Normalize and create sequences
normalized_data = fe.normalize_data(df_features)
sequence_length = 30
X, y = fe.create_sequences(df_features, sequence_length=sequence_length, target_col='Close')

In [6]:
# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

In [7]:
print("Input shape:", X_tensor.shape)
print("Target shape:", y_tensor.shape)

Input shape: torch.Size([303, 30, 27])
Target shape: torch.Size([303, 1])


In [8]:
# Create DataLoader
batch_size = 32
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


Initialize LSTM model

In [9]:
input_size = X_tensor.shape[2]  # number of features
hidden_size = 64
num_layers = 2
output_size = 1
dropout = 0.2
bidirectional = False
use_gru = False

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [ ]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout, bidirectional, use_gru).to(device)
print(model)


Loss & optimizer

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training loop

In [ ]:
num_epochs = 5  # for demo; increase for real training

model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for xb, yb in dataloader:
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        output = model(xb)
        loss = criterion(output, yb)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader):.6f}")


Inference and plotting

In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_tensor.to(device)).cpu().numpy()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y, label='Actual Close')
plt.plot(predictions, label='Predicted Close')
plt.title("BTC-USD LSTM Predictions")
plt.legend()
plt.show()


Save/load model with `torch.save(model.state_dict(), 'lstm.pth')` and `model.load_state_dict(torch.load('lstm.pth'))`.